# Synthesize a coloring book

In [4]:
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage

from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches

## Define keys and load models

In [5]:
# openAI key
openai.api_key = ""
# for azure openAI
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

## Inputs

In [11]:
topic = "anthropormophic zoo animals playing sports"
number_pages = 40
images_to_generate = 5
# upscale image
enhance_images = False

## Helper functions

In [7]:
# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" ideas for a coloring book on the topic of "+topic+"/n/nIdeas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# parse a string to list based on return characters
def parse_string(input_string):
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if not 'Create ' in line and not line == "":
            result.append(line.strip())
    return result

def generate(plain_text):
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

## Workflow

In [8]:
# Create page ideas given the desired topic
page_ideas = generate_coloring_book_page_descriptions(topic, number_pages)
print(page_ideas)

('Anthropomorphic Zoo Animals Playing Sports\n\n1. Elephant playing soccer\n2. Lion playing basketball\n3. Giraffe playing tennis\n4. Monkey playing baseball\n5. Hippopotamus playing hockey\n6. Gorilla playing football\n7. Rhinoceros playing lacrosse\n8. Cheetah playing badminton\n9. Camel playing golf\n10. Tiger playing volleyball\n11. Alligator playing rugby\n12. Polar bear playing cricket\n13. Zebra playing frisbee\n14. Bear playing handball\n15. Kangaroo playing water polo\n16. Wolf playing squash\n17. Fox playing softball\n18. Coyote playing dodgeball\n19. Porcupine playing croquet\n20. Vulture playing billiards\n21. Hawk playing horseshoes\n22. Ostrich playing archery\n23. Emu playing table tennis\n24. Beaver playing curling\n25. Dolphin playing track and field\n26. Walrus playing martial arts\n27. Seal playing kickball\n28. Sea lion playing bocce\n29. Penguin playing skittles\n30. Pelican playing bowling\n31. Flamingo playing beach volleyball\n32. Toucan playing roller hockey\n3

In [9]:
# parse the page ideas into a list for each page
pages = parse_string(page_ideas[0])
print(pages)

['Anthropomorphic Zoo Animals Playing Sports', '1. Elephant playing soccer', '2. Lion playing basketball', '3. Giraffe playing tennis', '4. Monkey playing baseball', '5. Hippopotamus playing hockey', '6. Gorilla playing football', '7. Rhinoceros playing lacrosse', '8. Cheetah playing badminton', '9. Camel playing golf', '10. Tiger playing volleyball', '11. Alligator playing rugby', '12. Polar bear playing cricket', '13. Zebra playing frisbee', '14. Bear playing handball', '15. Kangaroo playing water polo', '16. Wolf playing squash', '17. Fox playing softball', '18. Coyote playing dodgeball', '19. Porcupine playing croquet', '20. Vulture playing billiards', '21. Hawk playing horseshoes', '22. Ostrich playing archery', '23. Emu playing table tennis', '24. Beaver playing curling', '25. Dolphin playing track and field', '26. Walrus playing martial arts', '27. Seal playing kickball', '28. Sea lion playing bocce', '29. Penguin playing skittles', '30. Pelican playing bowling', '31. Flamingo p

In [ ]:
# create fun facts and images for each page
fun_facts = []
images = []
count = 0
# iterate for each desired page
for page in pages:
    prompt = "photo, anthropormophic "+page
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt).images[0]
        # enhance/upscale the generated image
        if enhance_images:
            enhanced_image = enhance_image(image)
            # create a coloring book image based on the generated image
            coloring_book_image = generate_coloring_page(enhanced_image)
        else:
            coloring_book_image = generate_coloring_page(image)
        # save the image with the format image_pageNumber_imageNumber as jpg
        coloring_book_image.save("image_"+str(count)+"_"+str(image_num)+".jpg")
    # add the fun facts for each page in a list
    fun_facts.append(generate_fun_facts(page))
    count = count + 1

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# write to a word document
document = Document()
document.add_heading('Coloring Book', 0)

p = document.add_paragraph('A coloring book about '+topic)
#p.add_run('bold').bold = True
#p.add_run(' and some ')
#p.add_run('italic.').italic = True

count = 0
for page in pages:
    document.add_page_break()
    document.add_heading('Heading, level 1', level=1)
    document.add_paragraph('Fun facts about '+pages[count], style='Intense Quote')
    document.add_picture("image_"+str(count)+"_0.jpg", width=Inches(5.0))
    document.add_paragraph(fun_facts[count])
    count += 1
#document.add_paragraph(
#    'first item in ordered list', style='List Number'
#)



#records = (
#    (3, '101', 'Spam'),
#    (7, '422', 'Eggs'),
#    (4, '631', 'Spam, spam, eggs, and spam')
#)

#table = document.add_table(rows=1, cols=3)
#hdr_cells = table.rows[0].cells
#hdr_cells[0].text = 'Qty'
#hdr_cells[1].text = 'Id'
#hdr_cells[2].text = 'Desc'
#for qty, id, desc in records:
#    row_cells = table.add_row().cells
#    row_cells[0].text = str(qty)
#    row_cells[1].text = id
#    row_cells[2].text = desc

document.save('demo.docx')